## 기업 데이터에 추가 데이터 붙이기

In [221]:
import pandas as pd

gi = pd.read_csv('../data/gi.csv')

In [222]:
gi = gi.iloc[:,1:]

In [223]:
gi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   회사명         533 non-null    object
 1   stock_code  533 non-null    int64 
 2   상장일         533 non-null    object
dtypes: int64(1), object(2)
memory usage: 12.6+ KB


In [224]:
gi

,회사명,stock_code,상장일
0,바이오노트,377740,2022-12-22
1,쏘카,403550,2022-08-22
2,수산인더스트리,126720,2022-08-01
3,LG에너지솔루션,373220,2022-01-27
4,카카오페이,377300,2021-11-03
...,...,...,...
528,싸이맥스,160980,2015-06-17
529,픽셀플러스,87600,2015-06-12
530,제노포커스,187420,2015-05-29
531,유지인트,195990,2015-04-13


In [225]:
# 1) KOSPI200 데이터 읽기 & 수익률 계산
ko200 = pd.read_csv('../notion_data/KOSPI200.csv', parse_dates=['날짜'])
ko200 = ko200.sort_values('날짜').reset_index(drop=True)
ko200['kospi200(-20)'] = ko200['종가'].shift(1) / ko200['종가'].shift(20) - 1

# 2) gi 데이터 읽기
gi = pd.read_csv('../data/gi.csv', parse_dates=['상장일'])
gi = gi.sort_values('상장일').reset_index(drop=True)

# 3) asof merge 로 상장일 기준 최근 거래일의 kospi200(-20) 가져오기
merged = pd.merge_asof(
    gi,
    ko200[['날짜', 'kospi200(-20)']].sort_values('날짜'),
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)
0         532    포시에스      189690 2015-02-11      -0.053391
1          68  세화아이엠씨      145210 2015-03-19       0.049828
2          67   엔에스쇼핑      138250 2015-03-27       0.058506
3         531    유지인트      195990 2015-04-13      -0.016436
4         530   제노포커스      187420 2015-05-29       0.028846


In [226]:
nasdaq = pd.read_csv('../notion_data/Nasdaq.csv', parse_dates=['Price'])
nasdaq = nasdaq.sort_values('Price').reset_index(drop=True)
nasdaq['nasdaq(-20)'] = nasdaq['Close'].shift(1) / nasdaq['Close'].shift(20) - 1

merged = pd.merge_asof(
    merged,
    nasdaq[['Price', 'nasdaq(-20)']].sort_values('Price'),
    left_on='상장일',
    right_on='Price',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['Price'])

# 4) 결과 확인
print(merged.head())



   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683


In [227]:

putcall = pd.read_csv('../notion_data/PutCall_ratio.csv',encoding='cp949',parse_dates=['일자'])
putcall = putcall.sort_values('일자').reset_index(drop=True)
putcall['putcall(-20)'] = putcall['P/C Ratio'].shift(1) / putcall['P/C Ratio'].shift(20) - 1

merged = pd.merge_asof(
    merged,
    putcall[['일자', 'putcall(-20)']].sort_values('일자'),
    left_on='상장일',
    right_on='일자',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['일자'])

# 4) 결과 확인
print(merged.head())



   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  
0      0.056075  
1     -0.338983  
2     -0.042553  
3     -0.070707  
4      0.324324  


In [228]:
vixcls = pd.read_csv('../notion_data/VIXCLS(미국).csv')
vixcls = vixcls.sort_values('observation_date').reset_index(drop=True)

# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
vixcls['observation_date'] = pd.to_datetime(vixcls['observation_date'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
vixcls = vixcls.sort_values('observation_date').reset_index(drop=True)

# 3) Perform the merge_asof
merged = pd.merge_asof(
    merged,
    vixcls[['observation_date', 'VIXCLS']],
    left_on='상장일',
    right_on='observation_date',
    direction='backward'
).drop(columns=['observation_date'])

print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  
0      0.056075   16.96  
1     -0.338983   14.07  
2     -0.042553   15.07  
3     -0.070707   13.94  
4      0.324324   13.84  


In [229]:
us10 = pd.read_csv('../notion_data/미국10년물국채금리채권수익률.csv')
us10 = us10.sort_values('날짜').reset_index(drop=True)
us10['미국10년물국채금리수익률(-20)'] = us10['종가'].shift(1) / us10['종가'].shift(20) - 1

# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
us10['날짜'] = pd.to_datetime(us10['날짜'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
us10 = us10.sort_values('날짜').reset_index(drop=True)

merged = pd.merge_asof(
    merged,
    us10[['날짜', '미국10년물국채금리수익률(-20)']].sort_values('날짜'),
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  
0      0.056075   16.96            0.080323  
1     -0.338983   14.07           -0.096136  
2     -0.042553   15.07            0.000000  
3     -0.070707   13.94           -0.060270  
4      0.324324   13.84            0.010890  


In [230]:
market_d1 = pd.read_csv('../notion_data/시장금리국고채3년(일별).csv',parse_dates=['변환'])
market_d1 = market_d1.sort_values('변환').reset_index(drop=True)

# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
market_d1['변환'] = pd.to_datetime(market_d1['변환'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
market_d1 = market_d1.sort_values('변환').reset_index(drop=True)

# 3) Perform the merge_asof
merged = pd.merge_asof(
    merged,
    market_d1[['변환', '원자료']],
    left_on='상장일',
    right_on='변환',
    direction='backward'
).drop(columns=['변환'])

merged.rename(columns={'원자료':'시장금리국고채3년'},inplace=True)
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  시장금리국고채3년  
0      0.056075   16.96            0.080323      2.051  
1     -0.338983   14.07           -0.096136      1.825  
2     -0.042553   15.07            0.000000      1.770  
3     -0.070707   13.94           -0.060270      1.732  
4      0.324324   13.84            0.010890      1.745  


In [231]:
ghksdbf = pd.read_csv('../notion_data/주요국통화.csv',parse_dates=['변환'])
ghksdbf = ghksdbf.sort_values('변환').reset_index(drop=True)
# remove commas (using regex=False for literal replace) and convert to float
ghksdbf['원자료'] = (
    ghksdbf['원자료']
    .str.replace(',', '', regex=False)
    .astype(float)
)
ghksdbf['환율변동률(-20)'] = ghksdbf['원자료'].shift(1) / ghksdbf['원자료'].shift(20) - 1


# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
ghksdbf['변환'] = pd.to_datetime(ghksdbf['변환'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
ghksdbf = ghksdbf.sort_values('변환').reset_index(drop=True)

merged = pd.merge_asof(
    merged,
    ghksdbf[['변환', '환율변동률(-20)']].sort_values('변환'),
    left_on='상장일',
    right_on='변환',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['변환'])

# merged.rename(columns={'원자료':'시장금리국고채3년'},inplace=True)
# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  시장금리국고채3년  환율변동률(-20)  
0      0.056075   16.96            0.080323      2.051    0.014332  
1     -0.338983   14.07           -0.096136      1.825    0.028860  
2     -0.042553   15.07            0.000000      1.770    0.003184  
3     -0.070707   13.94           -0.060270      1.732   -0.028163  
4      0.324324   13.84            0.010890      1.745    0.030247  


In [232]:
kospi = pd.read_csv('../notion_data/코스피지수.csv',parse_dates=['변환'])
kospi = kospi.sort_values('변환').reset_index(drop=True)
# remove commas (using regex=False for literal replace) and convert to float
kospi['원자료'] = (
    kospi['원자료']
    .str.replace(',', '', regex=False)
    .astype(float)
)
kospi['코스피변동률(-20)'] = kospi['원자료'].shift(1) / kospi['원자료'].shift(20) - 1


# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
kospi['변환'] = pd.to_datetime(kospi['변환'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
kospi = kospi.sort_values('변환').reset_index(drop=True)

merged = pd.merge_asof(
    merged,
    kospi[['변환', '코스피변동률(-20)']].sort_values('변환'),
    left_on='상장일',
    right_on='변환',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['변환'])

# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  시장금리국고채3년  환율변동률(-20)  \
0      0.056075   16.96            0.080323      2.051    0.014332   
1     -0.338983   14.07           -0.096136      1.825    0.028860   
2     -0.042553   15.07            0.000000      1.770    0.003184   
3     -0.070707   13.94           -0.060270      1.732   -0.028163   
4      0.324324   13.84            0.010890      1.745    0.030247   

   코스피변동률(-20)  
0     0.011601  
1     0.035859  
2     0.018511  
3     0.050535  
4    -0.017126  

In [233]:
kosdaq = pd.read_csv('../notion_data/코스닥지수.csv',parse_dates=['변환'])
kosdaq = kosdaq.sort_values('변환').reset_index(drop=True)
# remove commas (using regex=False for literal replace) and convert to float
kosdaq['원자료'] = (
    kosdaq['원자료']
    .str.replace(',', '', regex=False)
    .astype(float)
)
kosdaq['코스닥변동률(-20)'] = kosdaq['원자료'].shift(1) / kosdaq['원자료'].shift(20) - 1


# 1) Convert both columns to datetime
merged['상장일'] = pd.to_datetime(merged['상장일'])
kosdaq['변환'] = pd.to_datetime(kosdaq['변환'])

# 2) Sort both DataFrames by their date keys
merged = merged.sort_values('상장일').reset_index(drop=True)
kosdaq = kosdaq.sort_values('변환').reset_index(drop=True)

merged = pd.merge_asof(
    merged,
    kosdaq[['변환', '코스닥변동률(-20)']].sort_values('변환'),
    left_on='상장일',
    right_on='변환',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['변환'])

# 4) 결과 확인
print(merged.head())


   Unnamed: 0     회사명  stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스      189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨      145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑      138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트      195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스      187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  시장금리국고채3년  환율변동률(-20)  \
0      0.056075   16.96            0.080323      2.051    0.014332   
1     -0.338983   14.07           -0.096136      1.825    0.028860   
2     -0.042553   15.07            0.000000      1.770    0.003184   
3     -0.070707   13.94           -0.060270      1.732   -0.028163   
4      0.324324   13.84            0.010890      1.745    0.030247   

   코스피변동률(-20)  코스닥변동률(-20)  
0     0.011601     0.032708  
1     0.035859     0.030861  
2     0.018511     0.028772  
3 

## 외국인순매수, 원자재변동률 투자자별주식거래 추가해야 됨. (feat. 결측치 처리도 해야됨)

In [234]:
merged.isnull().sum()

Unnamed: 0             0
회사명                    0
stock_code             0
상장일                    0
kospi200(-20)          0
nasdaq(-20)            0
putcall(-20)           0
VIXCLS                10
미국10년물국채금리수익률(-20)     0
시장금리국고채3년              0
환율변동률(-20)             0
코스피변동률(-20)            1
코스닥변동률(-20)            0
dtype: int64

In [235]:
merged.head()

,Unnamed: 0,회사명,stock_code,상장일,kospi200(-20),nasdaq(-20),putcall(-20),VIXCLS,미국10년물국채금리수익률(-20),시장금리국고채3년,환율변동률(-20),코스피변동률(-20),코스닥변동률(-20)
0,532,포시에스,189690,2015-02-11,-0.053391,0.027060,0.056075,16.96,0.080323,2.051,0.014332,0.011601,0.032708
1,68,세화아이엠씨,145210,2015-03-19,0.049828,0.011804,-0.338983,14.07,-0.096136,1.825,0.028860,0.035859,0.030861
2,67,엔에스쇼핑,138250,2015-03-27,0.058506,-0.020181,-0.042553,15.07,0.000000,1.770,0.003184,0.018511,0.028772
3,531,유지인트,195990,2015-04-13,-0.016436,0.025498,-0.070707,13.94,-0.060270,1.732,-0.028163,0.050535,0.073371
4,530,제노포커스,187420,2015-05-29,0.028846,0.031683,0.324324,13.84,0.010890,1.745,0.030247,-0.017126,0.017538


In [236]:
merged[merged['회사명']=='안트로젠']

,Unnamed: 0,회사명,stock_code,상장일,kospi200(-20),nasdaq(-20),putcall(-20),VIXCLS,미국10년물국채금리수익률(-20),시장금리국고채3년,환율변동률(-20),코스피변동률(-20),코스닥변동률(-20)
67,481,안트로젠,65660,2016-02-15,0.300277,-0.075441,-0.145455,NaN,-0.14621,1.484,-0.009516,-0.042269,-0.113085


In [237]:
merged[merged['VIXCLS'].isna()]
#25.40
merged.loc[merged['회사명']=='안트로젠','VIXCLS']=25.40
#14.77
merged.loc[merged['회사명']=='피앤씨테크','VIXCLS']=14.77
#13.22
merged.loc[merged['회사명']=='제노레이','VIXCLS']=13.22
#20.80
merged.loc[merged['회사명']=='디알젬','VIXCLS']=20.80
#20.80
merged.loc[merged['회사명']=='싸이토젠','VIXCLS']=20.80
#20.74
merged.loc[merged['회사명']=='에코캡','VIXCLS']=20.74
#12.57
merged.loc[merged['회사명']=='펌텍코리아','VIXCLS']=12.57
#27.68
merged.loc[merged['회사명']=='위더스제약','VIXCLS']=27.68
#21.25
merged.loc[merged['회사명']=='제일전기공업','VIXCLS']=21.25
#27.75
merged.loc[merged['회사명']=='바이오에프디엔씨','VIXCLS']=27.75

In [238]:
merged[merged['코스피변동률(-20)'].isna()]
#0.008700
merged.loc[merged['회사명']=='포시에스','코스피변동률(-20)']=0.008700
#-0.063727
merged.loc[merged['회사명']=='핑거','코스피변동률(-20)']=-0.063727

In [239]:
merged[merged['코스닥변동률(-20)'].isna()]
#0.042736
merged.loc[merged['회사명']=='핑거','코스닥변동률(-20)']=0.042736

In [240]:
merged.isnull().sum()

Unnamed: 0            0
회사명                   0
stock_code            0
상장일                   0
kospi200(-20)         0
nasdaq(-20)           0
putcall(-20)          0
VIXCLS                0
미국10년물국채금리수익률(-20)    0
시장금리국고채3년             0
환율변동률(-20)            0
코스피변동률(-20)           0
코스닥변동률(-20)           0
dtype: int64

## ipo 데이터 병합하기

In [241]:
ipo = pd.read_csv('../data/backup_final_ratio.csv')

In [242]:
ipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   회사명_d1           657 non-null    object 
 1   stock_code       657 non-null    int64  
 2   업종               656 non-null    object 
 3   상장주선인            657 non-null    object 
 4   공모가 (원)          657 non-null    float64
 5   공모금액 (천원)        657 non-null    object 
 6   최초상장주식수 (주)      657 non-null    object 
 7   call             657 non-null    float64
 8   kospi            657 non-null    object 
 9   modified_esi     657 non-null    float64
 10  6개월 확약           657 non-null    float64
 11  3개월 확약           657 non-null    float64
 12  1개월 확약           657 non-null    float64
 13  15일 확약           657 non-null    float64
 14  미확약              657 non-null    float64
 15  수요예측_최종값         657 non-null    float64
 16  회사명_bs           657 non-null    object 
 17  corp_code       

In [243]:
# Option 1: get a list of column names
object_cols = ipo.select_dtypes(include=['object']).columns.tolist()
print(object_cols)


['회사명_d1', '업종', '상장주선인', '공모금액 (천원)', '최초상장주식수 (주)', 'kospi', '회사명_bs', '상장일', 'WICS_large', '우리사주조합', '기관투자자', '일반투자자']


In [244]:
cols = [
    '공모금액 (천원)',
    '최초상장주식수 (주)',
    'kospi',
    '우리사주조합',
    '기관투자자',
    '일반투자자'
]

In [245]:
# cols 리스트는 이미 정의되어 있다고 가정
# 1) 각 셀에 '-'인지 체크해서 Boolean DataFrame 생성
dash_mask = ipo[cols].eq('-')

# 2) 한 행이라도 True인(즉 '-'가 하나라도 있는) 행을 골라내는 mask
rows_with_dash = dash_mask.any(axis=1)

# 3) 실제로 해당 행들만 추출
ipo_with_dash = ipo[rows_with_dash]

# 4) 결과 확인
ipo_with_dash[cols]


,공모금액 (천원),최초상장주식수 (주),kospi,우리사주조합,기관투자자,일반투자자
4,"435,000,000","58,025,720","2,891.35",-,75.00,25.00
22,"113,741,798","27,674,406","3,143.09",-,75,25
34,"57,600,000","11,200,000","2,153.24",-,3.8,96.2
81,"9,000,000","5,625,900","2,429.67",-,75.00,25.00
83,"20,150,000","10,803,960","2,435.93",-,75,25
...,...,...,...,...,...,...
472,"19,800,000","11,718,545","2,108.22",-,0,100.0
480,"25,420,000","3,232,600","2,089.62",-,0,100.0
516,"16,800,000","6,782,000","2,472.37",-,0,100.0
566,"7,410,000","22,421,004","2,036.65",-,0,100.0


In [246]:
import numpy as np
import pandas as pd



ipo[cols] = (
    ipo[cols]
        .replace({',': ''}, regex=True)   # remove commas
        .replace('-', '0')       # swap '-' for the string '0'
        .astype(float)           # then cast the whole block to floats
)


In [247]:
ipo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 657 entries, 0 to 656
Data columns (total 65 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   회사명_d1           657 non-null    object 
 1   stock_code       657 non-null    int64  
 2   업종               656 non-null    object 
 3   상장주선인            657 non-null    object 
 4   공모가 (원)          657 non-null    float64
 5   공모금액 (천원)        657 non-null    float64
 6   최초상장주식수 (주)      657 non-null    float64
 7   call             657 non-null    float64
 8   kospi            657 non-null    float64
 9   modified_esi     657 non-null    float64
 10  6개월 확약           657 non-null    float64
 11  3개월 확약           657 non-null    float64
 12  1개월 확약           657 non-null    float64
 13  15일 확약           657 non-null    float64
 14  미확약              657 non-null    float64
 15  수요예측_최종값         657 non-null    float64
 16  회사명_bs           657 non-null    object 
 17  corp_code       

In [248]:
import pandas as pd

# Make sure both keys are the same dtype
merged['stock_code'] = merged['stock_code'].astype(str)
ipo['stock_code'] = ipo['stock_code'].astype(str)

# Perform the merge
merged_full = pd.merge(
    merged,
    ipo,
    on='stock_code',
    how='left',               # keep all rows in merged, add matches from ipo
    suffixes=('', '_ipo')     # if there are overlapping column names
)

# Inspect the result
print(merged_full.head())


   Unnamed: 0     회사명 stock_code        상장일  kospi200(-20)  nasdaq(-20)  \
0         532    포시에스     189690 2015-02-11      -0.053391     0.027060   
1          68  세화아이엠씨     145210 2015-03-19       0.049828     0.011804   
2          67   엔에스쇼핑     138250 2015-03-27       0.058506    -0.020181   
3         531    유지인트     195990 2015-04-13      -0.016436     0.025498   
4         530   제노포커스     187420 2015-05-29       0.028846     0.031683   

   putcall(-20)  VIXCLS  미국10년물국채금리수익률(-20)  시장금리국고채3년  ...  우리사주조합  기관투자자  \
0      0.056075   16.96            0.080323      2.051  ...    10.0    0.0   
1     -0.338983   14.07           -0.096136      1.825  ...    20.0    0.0   
2     -0.042553   15.07            0.000000      1.770  ...     0.0    0.0   
3     -0.070707   13.94           -0.060270      1.732  ...    18.5    0.0   
4      0.324324   13.84            0.010890      1.745  ...    14.9    0.0   

   일반투자자 target_1d target_2d target_7d  target_15d  target_1m  target_3m  \
0   

In [249]:
merged_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 533 entries, 0 to 532
Data columns (total 77 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   Unnamed: 0          533 non-null    int64         
 1   회사명                 533 non-null    object        
 2   stock_code          533 non-null    object        
 3   상장일                 533 non-null    datetime64[ns]
 4   kospi200(-20)       533 non-null    float64       
 5   nasdaq(-20)         533 non-null    float64       
 6   putcall(-20)        533 non-null    float64       
 7   VIXCLS              533 non-null    float64       
 8   미국10년물국채금리수익률(-20)  533 non-null    float64       
 9   시장금리국고채3년           533 non-null    float64       
 10  환율변동률(-20)          533 non-null    float64       
 11  코스피변동률(-20)         533 non-null    float64       
 12  코스닥변동률(-20)         533 non-null    float64       
 13  회사명_d1              533 non-null    object        

In [250]:
# merged_full.to_csv('../data/pre_data.csv')

In [251]:
merged_full.isna().sum()[merged_full.isna().sum() > 0]

업종                 1
wc_sales          10
de_ratio          14
equity_ratio      14
cs_te             14
op_margin         10
net_margin        10
roe               14
roe_sign_dummy    14
sales_equity      14
dtype: int64

## 원자재 종가

In [252]:
dnjs = pd.read_csv('../notion_data/원자재_종가.csv')

In [253]:
dnjs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3652 entries, 0 to 3651
Data columns (total 37 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   날짜                3652 non-null   object 
 1   런던_가스오일  3652 non-null   float64
 2   난방유          3652 non-null   float64
 3   탄소배출권     3652 non-null   float64
 4   브렌트유         3652 non-null   float64
 5   가솔린          3652 non-null   float64
 6   천연가스        3652 non-null   float64
 7   WTI유             3652 non-null   float64
 8   금               3652 non-null   float64
 9   은               3652 non-null   float64
 10  백금            3652 non-null   float64
 11  주석             3652 non-null   float64
 12  납               3652 non-null   float64
 13  아연             3652 non-null   float64
 14  알루미늄        3652 non-null   float64
 15  니켈             3652 non-null   float64
 16  구리_영국       3652 non-null   float64


In [254]:
dnjs['날짜'] = pd.to_datetime(dnjs['날짜'])
dnjs = dnjs.sort_values('날짜').reset_index(drop=True)

# '날짜'를 인덱스로 설정
df = dnjs.set_index('날짜')

# 일별 수익률 계산 (pct_change는 전일 대비 퍼센트 변동)
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# (shift, 라벨) 쌍 정의: 
# shift=1 → -1~-5일, shift=6 → -6~-10일, shift=11 → -11~-15일, shift=16 → -16~-20일
windows = [
    (1, '(-1~-5)'),
    (6, '(-6~-10)'),
    (11, '(-11~-15)'),
    (16, '(-16~-20)')
]

# 각 컬럼에 대해 네 구간별 표준편차 계산
for col in df.columns:
    for shift, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(shift)              # 과거 shift 일만큼 뒤로
            .rolling(window=5)         # 5일간
            .std()                     # 표준편차
        )

# 인덱스를 '날짜' 컬럼으로 되돌리고
result = result.reset_index()

# 결과 확인
print(result.head())
# 필요시 to_csv 등으로 저장
# result.to_csv('원자재_표준편차_윈도우.csv', index=False)


          날짜  런던_가스오일(-1~-5)  런던_가스오일(-6~-10)  \
0 2015-01-02                      NaN                       NaN   
1 2015-01-03                      NaN                       NaN   
2 2015-01-04                      NaN                       NaN   
3 2015-01-05                      NaN                       NaN   
4 2015-01-06                      NaN                       NaN   

   런던_가스오일(-11~-15)  런던_가스오일(-16~-20)  난방유(-1~-5)  \
0                        NaN                        NaN              NaN   
1                        NaN                        NaN              NaN   
2                        NaN                        NaN              NaN   
3                        NaN                        NaN              NaN   
4                        NaN                        NaN              NaN   

   난방유(-6~-10)  난방유(-11~-15)  난방유(-16~-20)  \
0               NaN                NaN                NaN   
1               Na

C:\Users\Owner\AppData\Local\Temp\ipykernel_3020\1881168302.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[f"{col}{label}"] = (
C:\Users\Owner\AppData\Local\Temp\ipykernel_3020\1881168302.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  result[f"{col}{label}"] = (
C:\Users\Owner\AppData\Local\Temp\ipykernel_3020\1881168302.py:25: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once u

In [255]:
result.head(30)

,날짜,런던_가스오일(-1~-5),런던_가스오일(-6~-10),런던_가스오일(-11~-15),런던_가스오일(-16~-20),난방유(-1~-5),난방유(-6~-10),난방유(-11~-15),난방유(-16~-20),탄소배출권(-1~-5),...,런던 코코아 선물 과거 데이터(-11~-15),런던 코코아 선물 과거 데이터(-16~-20),육우(-1~-5),육우(-6~-10),육우(-11~-15),육우(-16~-20),미국_대두(-1~-5),미국_대두(-6~-10),미국_대두(-11~-15),미국_대두(-16~-20)
0,2015-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2015-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2015-01-08,0.018479,NaN,NaN,NaN,0.013080,NaN,NaN,NaN,0.009809,...,NaN,NaN,0.004605,NaN,NaN,NaN,0.015872,NaN,NaN,NaN
7,2015-01-09,0.019862,NaN,NaN,NaN,0.013718,NaN,NaN,NaN,0.010712,...,NaN,NaN,0.004616,NaN,NaN,NaN,0.017300,NaN,NaN,NaN
8,2015-01-10,0.017930,NaN,NaN,NaN,0.012576,NaN,NaN,NaN,0.010763,...,NaN,NaN,0.008477,NaN,NaN,NaN,0.016755,NaN,NaN,NaN
9,2015-01-11,0.012189,NaN,NaN,NaN,0.009127,NaN,NaN,NaN,0.010485,...,NaN,NaN,0.006987,NaN,NaN,NaN,0.006837,NaN,NaN,NaN


In [256]:
merged_full

,Unnamed: 0,회사명,stock_code,상장일,kospi200(-20),nasdaq(-20),putcall(-20),VIXCLS,미국10년물국채금리수익률(-20),시장금리국고채3년,...,우리사주조합,기관투자자,일반투자자,target_1d,target_2d,target_7d,target_15d,target_1m,target_3m,target_6m
0,532,포시에스,189690,2015-02-11,-0.053391,0.027060,0.056075,16.96,0.080323,2.051,...,10.00,0.00,90.0,1,1,1,1,1,1,0
1,68,세화아이엠씨,145210,2015-03-19,0.049828,0.011804,-0.338983,14.07,-0.096136,1.825,...,20.00,0.00,80.0,1,1,1,1,1,0,0
2,67,엔에스쇼핑,138250,2015-03-27,0.058506,-0.020181,-0.042553,15.07,0.000000,1.770,...,0.00,0.00,100.0,0,0,1,0,0,0,0
3,531,유지인트,195990,2015-04-13,-0.016436,0.025498,-0.070707,13.94,-0.060270,1.732,...,18.50,0.00,81.5,1,1,1,1,1,1,1
4,530,제노포커스,187420,2015-05-29,0.028846,0.031683,0.324324,13.84,0.010890,1.745,...,14.90,0.00,85.1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,73,기가비스,420770,2023-05-24,-0.020188,0.059548,-0.094118,20.03,0.073440,3.378,...,5.96,69.04,25.0,1,1,1,1,1,1,1
529,72,진영,285800,2023-06-01,0.041520,0.075670,-0.132653,15.65,0.061426,3.478,...,0.00,75.00,25.0,1,1,1,1,1,0,0
530,71,나라셀라,405920,2023-06-02,-0.049039,0.094814,-0.135135,14.60,0.026804,3.436,...,3.00,72.00,25.0,0,0,1,0,0,0,0
531,70,마녀공장,439090,2023-06-08,-0.047255,0.064881,0.056180,13.65,0.095293,3.543,...,0.00,75.00,25.0,1,1,1,1,1,1,1


In [257]:
# 1) Convert both columns to datetime
merged_full['상장일'] = pd.to_datetime(merged_full['상장일'])
result['날짜'] = pd.to_datetime(result['날짜'])

# 2) Sort both DataFrames by their date keys
merged_full = merged_full.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('날짜').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='날짜',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['날짜'])

In [258]:
merged_full.isna().sum()[merged_full.isna().sum() > 0]

업종                 1
wc_sales          10
de_ratio          14
equity_ratio      14
cs_te             14
op_margin         10
net_margin        10
roe               14
roe_sign_dummy    14
sales_equity      14
dtype: int64

In [259]:
# merged_full.to_csv('../data/pre_data.csv')

## 코스닥 지수 

In [260]:
kosdaq = pd.read_csv('../notion_data/코스닥지수.csv')

In [261]:
kosdaq['원자료'] = pd.to_numeric(
    kosdaq['원자료'].str.replace(',', ''), 
    errors='coerce'
)
kosdaq.rename(columns={'원자료':'코스닥수익률표준편차'},inplace=True)
kosdaq.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2468 entries, 0 to 2467
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   변환          2468 non-null   object 
 1   코스닥수익률표준편차  2468 non-null   float64
dtypes: float64(1), object(1)
memory usage: 38.7+ KB


In [262]:
kosdaq['변환'] = pd.to_datetime(kosdaq['변환'])
kosdaq = kosdaq.sort_values('변환').reset_index(drop=True)

# '변환'를 인덱스로 설정
df = kosdaq.set_index('변환')

# 일별 수익률 계산 (pct_change는 전일 대비 퍼센트 변동)
returns = df.pct_change()

# 결과 저장용 DataFrame 생성
result = pd.DataFrame(index=df.index)

# (shift, 라벨) 쌍 정의: 
# shift=1 → -1~-5일, shift=6 → -6~-10일, shift=11 → -11~-15일, shift=16 → -16~-20일
windows = [
    (1, '(-1~-5)'),
    (6, '(-6~-10)'),
    (11, '(-11~-15)'),
    (16, '(-16~-20)')
]

# 각 컬럼에 대해 네 구간별 표준편차 계산
for col in df.columns:
    for shift, label in windows:
        result[f"{col}{label}"] = (
            returns[col]
            .shift(shift)              # 과거 shift 일만큼 뒤로
            .rolling(window=5)         # 5일간
            .std()                     # 표준편차
        )

# 인덱스를 '변환' 컬럼으로 되돌리고
result = result.reset_index()

# 결과 확인
print(result.head())
# 필요시 to_csv 등으로 저장
# result.to_csv('원자재_표준편차_윈도우.csv', index=False)


          변환  코스닥수익률표준편차(-1~-5)  코스닥수익률표준편차(-6~-10)  코스닥수익률표준편차(-11~-15)  \
0 2015-01-02                NaN                 NaN                  NaN   
1 2015-01-05                NaN                 NaN                  NaN   
2 2015-01-06                NaN                 NaN                  NaN   
3 2015-01-07                NaN                 NaN                  NaN   
4 2015-01-08                NaN                 NaN                  NaN   

   코스닥수익률표준편차(-16~-20)  
0                  NaN  
1                  NaN  
2                  NaN  
3                  NaN  
4                  NaN  


In [263]:
result.head(30)

,변환,코스닥수익률표준편차(-1~-5),코스닥수익률표준편차(-6~-10),코스닥수익률표준편차(-11~-15),코스닥수익률표준편차(-16~-20)
0,2015-01-02,NaN,NaN,NaN,NaN
1,2015-01-05,NaN,NaN,NaN,NaN
2,2015-01-06,NaN,NaN,NaN,NaN
3,2015-01-07,NaN,NaN,NaN,NaN
4,2015-01-08,NaN,NaN,NaN,NaN
5,2015-01-09,NaN,NaN,NaN,NaN
6,2015-01-12,0.007087,NaN,NaN,NaN
7,2015-01-13,0.006091,NaN,NaN,NaN
8,2015-01-14,0.006984,NaN,NaN,NaN
9,2015-01-15,0.007006,NaN,NaN,NaN


In [264]:
# 1) Convert both columns to datetime
merged_full['상장일'] = pd.to_datetime(merged_full['상장일'])
result['변환'] = pd.to_datetime(result['변환'])

# 2) Sort both DataFrames by their date keys
merged_full = merged_full.sort_values('상장일').reset_index(drop=True)
result = result.sort_values('변환').reset_index(drop=True)

merged_full = pd.merge_asof(
    merged_full,
    result,
    left_on='상장일',
    right_on='변환',
    direction='backward'   # 상장일 당일 또는 그 이전의 가장 가까운 거래일
).drop(columns=['변환'])

In [265]:
# merged_full.to_csv('../data/pre_data.csv')

In [266]:
merged_full.isna().sum()[merged_full.isna().sum() > 0]

업종                 1
wc_sales          10
de_ratio          14
equity_ratio      14
cs_te             14
op_margin         10
net_margin        10
roe               14
roe_sign_dummy    14
sales_equity      14
dtype: int64